In [12]:
import pandas as pd
import numpy as np


Let's check what the number of anime that have been loaded:


In [13]:
df = pd.read_csv('anime_data.csv')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2149 entries, 0 to 2148
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            2149 non-null   object
 1   title         2149 non-null   object
 2   startDate     2149 non-null   object
 3   popularity    2149 non-null   object
 4   favourites    2149 non-null   object
 5   tags          2149 non-null   object
 6   genres        2149 non-null   object
 7   averageScore  2149 non-null   object
 8   duration      2149 non-null   object
 9   description   2149 non-null   object
 10  studios       2149 non-null   object
dtypes: object(11)
memory usage: 184.8+ KB
None
